# Simulações de Monte Carlo

### Importar bibliotecas

In [375]:
import numpy as np
import plotly.express as px
import pandas as pd

### Entrada de dados

In [376]:

# Dados das usinas
DUSI = np.array([
    [1, 2, 20, 10],
    [2, 1, 30, 15]
])


# Carga a ser atendida
PLOAD = 40  # MW


# Patamares de carga
#                   Patamar   Nível(%)    Horas
DLOAD = np.array([  [ 1,        100,        3  ],
                    [ 2,        80,         9  ],
                    [ 3,        50,         12 ]
])


### Interpretar dados de entrada 

In [377]:
usinas = DUSI[:, 0]
n_unidades = DUSI[:, 1]
pot = DUSI[:, 2]
FOR = DUSI[:, 3] / 100

n_usinas = len(usinas)
capacidade_usinas = n_unidades * pot
capacidade_total = sum(capacidade_usinas)  # total do sistema


# Sort DLOAD by the third column (ascending order)
DLOAD = DLOAD[DLOAD[:, 2].argsort()]

patamares = DLOAD[:, 0]
niveis = DLOAD[:, 1] / 100
horas_dia = DLOAD[:, 2]
total_horas = sum(horas_dia)
p_horas = horas_dia / total_horas  # proporção de horas em cada patamar

# número de patamares
n_patamares = len(patamares)

p_horas_acumulado = np.zeros(n_patamares)
for patamar in range(n_patamares):
    p_horas_acumulado[patamar] = np.sum(p_horas[:patamar + 1])

# lista com os patamares convertidos em string
patamares_str = [str(int(patamar)) for patamar in patamares]

# print('Patamares de carga: ', patamares)
# print('Níveis de carga: ', niveis)
# print('Horas por dia: ', horas_dia)
# print('Proporção de horas: ', p_horas)
# print('Proporção de horas acumulado: ', p_horas_acumulado)
# print('Patamares em string: ', patamares_str)


### Plotar gráfico com a proporção acumulada

In [378]:

df = pd.DataFrame({
    'Patamar': [' '] + patamares_str + ['  '],
    'Proporção de horas': [0] + list(p_horas_acumulado) + [1]
})

fig = px.line(df, x='Patamar', y='Proporção de horas', title='Proporção de horas por patamar',line_shape='hv')

# formatação do gráfico
altura = 450
fig.update_layout(height=altura, width=altura*1.5)
fig.update_xaxes(tickvals=[0] + list(np.sort(patamares) + 0.5) + [n_patamares], ticktext=[' '] + patamares_str + ['  '])
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1))
fig.update_xaxes(showgrid=False)
fig.show()

### **Algoritmo Principal**

In [379]:

tol = 0.01
NSmax = 1e7
NSmin = 500
beta_LOLP = tol + 1
NS = 0
soma_LOLP = 0
soma_quad_LOLP = 0
soma_EPNS = 0

while beta_LOLP > tol and NS < NSmax:
    NS += 1
    capacidade = capacidade_total

    U_carga = np.random.rand()

    carga = PLOAD
    for patamar in range(n_patamares):
        if U_carga < p_horas_acumulado[patamar]:
            carga = PLOAD * niveis[patamar]
            break


    for usi in range(n_usinas):
        for unid in range(n_unidades[usi]):
            Ui = np.random.rand()
            if Ui < FOR[usi]:
                capacidade -= pot[usi]

    if capacidade < carga:
        # LOLP
        soma_LOLP += 1
        soma_quad_LOLP += 1 ** 2  # p/ convergencia

        # EPNS
        soma_EPNS += (carga - capacidade)

    v_esperado_LOLP = soma_LOLP / NS  # --> RESPOSTA
    v_esperado_EPNS = soma_EPNS / NS

    # convergencia
    if NS > NSmin:
        variancia_LOLP = (soma_quad_LOLP - NS * v_esperado_LOLP**2) / (NS - 1)
        variancia_do_valor_esperado_LOLP = variancia_LOLP / NS
        beta_LOLP = np.sqrt(variancia_do_valor_esperado_LOLP) / v_esperado_LOLP
        if np.isnan(beta_LOLP):
            beta_LOLP = np.inf


### Resultados 

In [380]:
precisao = 5

print("LOLP: ", round(v_esperado_LOLP, precisao),
    "\nEPNS: ", round(v_esperado_EPNS, precisao) )



LOLP:  0.01893 
EPNS:  0.24222


#### Convergência

In [381]:
print('NS =', NS, 
'\nbeta_LOLP =', round(beta_LOLP, precisao))

NS = 518330 
beta_LOLP = 0.01
